# Pitch Detection Algorithm (PDA) by using the Zero Crossing Rate (ZCR)

Zero-crossing rate is actually a very simple algorithm, but it assumes that the source is monotonous. It counts the number of zero-crossings and every 2 zero-crossings are exactly one period. By that information, it is possible to compute the frequency.

Let $R$ be the frames per second and let $F$ be the frames per buffer. Then, one frame is $\frac{1}{R}$ seconds and every buffer takes $\frac{F}{R}$ seconds. Let $Z$ be the number of zero-crossings in the buffer, then there are $\frac{Z}{2}$ periods and that means that the fundamental frequency $f_0 = \frac{Z \cdot R}{2 \cdot F}$.

In [3]:
import pyaudio
from IPython.display import display, clear_output, HTML
%matplotlib inline

import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

from pitch_detection.utilities import *
from pitch_detection.algorithms import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
global keep_going
keep_going = True

audio = pyaudio.PyAudio()

frames_per_buffer = 8 * 1024
frames_per_second = 44100
stream = audio.open(format=pyaudio.paInt16,
                    channels=1,
                    frames_per_buffer=frames_per_buffer,
                    rate=frames_per_second,
                    input=True)

def process_audio(chunk_bytes):
    """Small helper function for converting a stream of bytes to the fundamental frequency and visualizing it."""
    freq = compute_freq_from_zerocrossings(buffer=chunk_bytes, frames_per_second=frames_per_second)
    tone = convert_freq_to_tone(freq) + " (" + str(int(freq)) + " Hz)"
    html = '<span style="background-color: rgb(0, {:d}, 0); padding: 20px 0; width: 100px; text-align: center; display: inline-block;">{:s}</span>'.format(int(35 * (np.log(freq))), tone)
    clear_output(wait=True)
    display(HTML(html))

process_audio(stream.read(frames_per_buffer))
while keep_going:
    try:
        process_audio(stream.read(frames_per_buffer))
    except KeyboardInterrupt:
        keep_going = False
    except:
        pass
    
stream.stop_stream()
stream.close()

audio.terminate()